In [24]:
import os
import cv2
import glob
import shutil
from sklearn.model_selection import train_test_split

In [25]:
IMG_SIZE = (640, 640)

def normalize_and_convert_images(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    images = glob.glob(os.path.join(input_dir, "*.jpg")) + glob.glob(os.path.join(input_dir, "*.png"))    
    for img_path in images:
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if img is None:
            print(f"⚠️ No se pudo leer la imagen: {img_path}")
            continue
        img_resized = cv2.resize(img, IMG_SIZE, interpolation=cv2.INTER_AREA)
        img_name = os.path.splitext(os.path.basename(img_path))[0] + ".jpg"  # Convertir todo a JPG
        cv2.imwrite(os.path.join(output_dir, img_name), img_resized, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

    print(f"✅ {len(images)} imágenes normalizadas y convertidas a JPG en {output_dir}")



In [26]:
raw_dir = "C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/raw"
normalized_dir = "C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/normalized"
for class_name in os.listdir(raw_dir):
    class_path = os.path.join(raw_dir, class_name)
    
    if os.path.isdir(class_path):
        output_class_path = os.path.join(normalized_dir, class_name)
        normalize_and_convert_images(class_path, output_class_path)


✅ 3215 imágenes normalizadas y convertidas a JPG en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/normalized\freshapples
✅ 3360 imágenes normalizadas y convertidas a JPG en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/normalized\freshbanana
✅ 1854 imágenes normalizadas y convertidas a JPG en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/normalized\freshoranges
✅ 806 imágenes normalizadas y convertidas a JPG en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/normalized\freshpotato
✅ 2113 imágenes normalizadas y convertidas a JPG en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/normalized\freshtomato
✅ 4236 imágenes normalizadas y convertidas a JPG en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/normalized\rottenapples
✅ 3832 imágenes normalizadas y convertidas a JPG en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/normalized\rottenbanana
✅ 1998 imágenes normalizadas y convertidas a JPG en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/norma

In [16]:
import os
import shutil
import glob
from sklearn.model_selection import train_test_split
import random

In [27]:
normalized_dir = "C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/normalized"
processed_dir = "C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/processed"

train_dir = os.path.join(processed_dir, "images/train")
val_dir = os.path.join(processed_dir, "images/val")

# Crear carpetas si no existen
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Obtener todas las imágenes con sus etiquetas
all_images = []
for class_name in os.listdir(normalized_dir):
    class_path = os.path.join(normalized_dir, class_name)
    if os.path.isdir(class_path):
        images = glob.glob(os.path.join(class_path, "*.jpg"))
        all_images.extend([(img, class_name) for img in images])

# Separar imágenes en entrenamiento (80%) y validación (20%)
images, labels = zip(*all_images)
train_images, val_images, _, _ = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

# Mover imágenes a sus respectivas carpetas
for img_path in train_images:
    shutil.copy(img_path, train_dir)

for img_path in val_images:
    shutil.copy(img_path, val_dir)

print(f"✅ Total imágenes de entrenamiento: {len(train_images)}")
print(f"✅ Total imágenes de validación: {len(val_images)}")


✅ Total imágenes de entrenamiento: 19811
✅ Total imágenes de validación: 4953


In [ ]:
# Directorios de etiquetas
labels_train_dir = os.path.join(processed_dir, "labels/train")
labels_val_dir = os.path.join(processed_dir, "labels/val")

os.makedirs(labels_train_dir, exist_ok=True)
os.makedirs(labels_val_dir, exist_ok=True)

# Mapeo de clases a números
class_mapping = {
    "freshapples": 0, "rottenapples": 1,
    "freshbanana": 2, "rottenbanana": 3,
    "freshoranges": 4, "rottenoranges": 5,
    "freshpotato": 6, "rottenpotato": 7,
    "freshtomato": 8, "rottentomato": 9
}

# Función para crear etiquetas
def generate_labels(image_paths, labels_dir):
    for img_path in image_paths:
        class_name = os.path.basename(os.path.dirname(img_path))  # Obtener la clase desde la carpeta
        class_id = class_mapping[class_name]  # Obtener ID de clase
        
        # Crear nombre del archivo de etiqueta
        label_file = os.path.splitext(os.path.basename(img_path))[0] + ".txt"
        label_path = os.path.join(labels_dir, label_file)
        
        # Escribir la etiqueta en formato YOLO
        with open(label_path, "w") as f:
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")  # Bounding box ocupa toda la imagen

# Generar etiquetas para entrenamiento y validación
generate_labels(train_images, labels_train_dir) 
generate_labels(val_images, labels_val_dir)

print("✅ Etiquetas YOLO generadas correctamente.")


✅ Etiquetas YOLO generadas correctamente.


In [ ]:
IMG_SIZE = (640, 640)

# Función para detectar el bounding box
def get_bounding_box(image):
    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Aplicar un umbral para detectar el objeto (fondo blanco es ignorado)
    _, threshold = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

    # Encontrar contornos (usamos la técnica de contorno para identificar el objeto)
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        # Obtener el contorno más grande (asumimos que es el objeto principal)
        c = max(contours, key=cv2.contourArea)
        # Obtener el rectángulo delimitador (x, y, w, h)
        x, y, w, h = cv2.boundingRect(c)
        return (x, y, w, h)
    return None

# Función para normalizar y convertir imágenes
def normalize_and_convert_images(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    images = glob.glob(os.path.join(input_dir, "*.jpg")) + glob.glob(os.path.join(input_dir, "*.png"))
    
    print(f"Imágenes encontradas: {len(images)} en {input_dir}")  # Verificar si hay imágenes
    
    for img_path in images:
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if img is None:
            print(f"⚠️ No se pudo leer la imagen: {img_path}")
            continue
        img_resized = cv2.resize(img, IMG_SIZE, interpolation=cv2.INTER_AREA)
        img_name = os.path.splitext(os.path.basename(img_path))[0] + ".jpg"  # Convertir todo a JPG
        cv2.imwrite(os.path.join(output_dir, img_name), img_resized, [int(cv2.IMWRITE_JPEG_QUALITY), 95])

    print(f"✅ {len(images)} imágenes normalizadas y convertidas a JPG en {output_dir}")

# Función para crear etiquetas en formato YOLO
def generate_labels(image_paths, labels_dir, class_mapping):
    for img_path in image_paths:
        img = cv2.imread(img_path)
        bbox = get_bounding_box(img)
        
        if bbox:
            x, y, w, h = bbox
            # Normalizar las coordenadas
            h_img, w_img, _ = img.shape
            x_center = (x + w / 2) / w_img
            y_center = (y + h / 2) / h_img
            width = w / w_img
            height = h / h_img
            
            class_name = os.path.basename(os.path.dirname(img_path))  # Obtener la clase desde la carpeta
            class_id = class_mapping[class_name]  # Obtener ID de clase
            
            # Crear nombre del archivo de etiqueta
            label_file = os.path.splitext(os.path.basename(img_path))[0] + ".txt"
            label_path = os.path.join(labels_dir, label_file)
            
            # Escribir la etiqueta en formato YOLO
            with open(label_path, "w") as f:
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Rutas de entrada y salida
raw_dir = "C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/raw"
normalized_dir = "C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/normalized"
processed_dir = "C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/processed"

# Crear las carpetas de salida
train_dir = os.path.join(processed_dir, "images/train")
val_dir = os.path.join(processed_dir, "images/val")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Mapeo de clases a números
class_mapping = {
    "Fresh_Tomato": 2, "Rotten_Tomato": 5, "Fresh_Banana": 0, "Rotten_Banana": 3,
    "Fresh_Orange": 1, "Rotten_Orange": 4
}


# Normalizar las imágenes
for class_name in os.listdir(raw_dir):
    class_path = os.path.join(raw_dir, class_name)
    if os.path.isdir(class_path):
        output_class_path = os.path.join(normalized_dir, class_name)
        normalize_and_convert_images(class_path, output_class_path)

# Obtener todas las imágenes con sus etiquetas
all_images = []
for class_name in os.listdir(normalized_dir):
    class_path = os.path.join(normalized_dir, class_name)
    if os.path.isdir(class_path):
        images = glob.glob(os.path.join(class_path, "*.jpg"))
        all_images.extend([(img, class_name) for img in images])

# Verifica que all_images tenga elementos
print(f"Total de imágenes encontradas para procesar: {len(all_images)}")

# Si no hay imágenes, asegúrate de revisar el directorio 'normalized_dir'
if len(all_images) == 0:
    print("⚠️ No se encontraron imágenes para procesar. Revisa el directorio 'normalized_dir'.")

# Separar imágenes en entrenamiento (80%) y validación (20%)
images, labels = zip(*all_images)
train_images, val_images, _, _ = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

# Mover imágenes a sus respectivas carpetas
for img_path in train_images:
    shutil.copy(img_path, train_dir)

for img_path in val_images:
    shutil.copy(img_path, val_dir)

print(f"✅ Total imágenes de entrenamiento: {len(train_images)}")
print(f"✅ Total imágenes de validación: {len(val_images)}")

# Directorios de etiquetas
labels_train_dir = os.path.join(processed_dir, "labels/train")
labels_val_dir = os.path.join(processed_dir, "labels/val")
os.makedirs(labels_train_dir, exist_ok=True)
os.makedirs(labels_val_dir, exist_ok=True)

# Generar etiquetas para entrenamiento y validación
generate_labels(train_images, labels_train_dir, class_mapping)
generate_labels(val_images, labels_val_dir, class_mapping)

print("✅ Etiquetas YOLO generadas correctamente.")


Imágenes encontradas: 0 en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/raw\archive
✅ 0 imágenes normalizadas y convertidas a JPG en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/normalized\archive
Imágenes encontradas: 2468 en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/raw\Fresh_Banana
✅ 2468 imágenes normalizadas y convertidas a JPG en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/normalized\Fresh_Banana
Imágenes encontradas: 1466 en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/raw\Fresh_Orange
✅ 1466 imágenes normalizadas y convertidas a JPG en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/normalized\Fresh_Orange
Imágenes encontradas: 1858 en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/raw\Fresh_Tomato
✅ 1858 imágenes normalizadas y convertidas a JPG en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/normalized\Fresh_Tomato
Imágenes encontradas: 2932 en C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/raw\Rotten_Banana
✅ 2932 i

In [ ]:
import yaml
import os

# Directorios de imágenes
train_img_dir = "C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/processed/images/train"
val_img_dir = "C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/processed/images/val"

# Mapeo de clases a nombres
class_mapping = {
    "Fresh_Tomato": 2, "Rotten_Tomato": 5, "Fresh_Banana": 0, "Rotten_Banana": 3,
    "Fresh_Orange": 1, "Rotten_Orange": 4
}



# Crear archivo YAML
yaml_data = {
    'train': train_img_dir,
    'val': val_img_dir,
    'nc': len(class_mapping),  # Número de clases
    'names': list(class_mapping.values())  # Lista con los nombres de las clases
}

# Guardar el archivo YAML
yaml_file = "C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/processed/dataset.yaml"
with open(yaml_file, 'w') as file:
    yaml.dump(yaml_data, file, default_flow_style=False, allow_unicode=True)

print(f"✅ Archivo YAML generado correctamente: {yaml_file}")


✅ Archivo YAML generado correctamente: C:/Users/marck/OneDrive/Documentos/Proyecto_ia/data/processed/dataset.yaml
